# Import packages

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_probability as tfp
import random
import numpy as np
tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions

from numpy import expand_dims, mean, ones
from numpy.random import randn, randint
from tensorflow.keras import backend
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.constraints import Constraint
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from netCDF4 import Dataset



# Dataset

### Generate synthetic dataset

In [66]:
seed = 7
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
A = 2*tfd.Uniform().sample(1000)
X = tfd.InverseGamma(concentration =1.5, scale = 0.6 ).sample(1000)
R1 = A*X
train_dataset = R1[:250]
train_dataset = tf.reshape(train_dataset,[250,1])
print(tf.reduce_max(train_dataset))

eval_dataset = R1[250:]
eval_dataset=tf.reshape(eval_dataset,[750,1])
print(tf.reduce_max(eval_dataset))

tf.Tensor(17.603828, shape=(), dtype=float32)
tf.Tensor(25.11976, shape=(), dtype=float32)


In [67]:
def alphas_function(x):
    res = tf.convert_to_tensor([6. -3*tf.minimum(1.,0.5/x), 6. -3*tf.minimum(1.,0.5/x) , 3.+ 3*tf.minimum(1,0.5/x), 3.+ 3*tf.minimum(1,0.5/x), 3.+ 3*tf.minimum(1,0.5/x)])
    return(tf.transpose(res))


alphas          = alphas_function(train_dataset[:,0])
angle_train_dep = tfd.Dirichlet(alphas).sample()
alphas_eval     = alphas_function(eval_dataset[:,0])
angle_eval_dep = tfd.Dirichlet(alphas_eval).sample()

In [ ]:
n_train_data = train_dataset*angle_train_dep

### Danube discharge measurement dataset

In [2]:
ncfile = Dataset('/home/nlafon/These/4Dvarnetstochastic/Danube_river_network/Dataset_danube.nc',"r")
L=[]
for i in range(31):
    L.append(ncfile['S'+str(i+1)][:].reshape(18244,1))
        
dataset = np.concatenate((L[0],L[1],L[2],L[3],L[4],L[5],L[6],L[7],L[8],L[9],L[10],L[11],L[12],L[13],L[14],L[15],L[16],L[17],L[18],L[19],L[20],L[21],L[22],L[23],L[24],L[25],L[26],L[27],L[28],L[29],L[30]),axis=1)
print(dataset.shape)


n_train_data = tf.convert_to_tensor(dataset[::25,22:27]/100)
print(n_train_data)

(18244, 31)


2023-01-19 15:35:53.360251: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-19 15:35:53.362502: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-19 15:35:53.447817: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 715.34GiB/s
2023-01-19 15:35:53.448046: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:73:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.68GiB deviceMemoryBandwidth: 715.34GiB/s
2023-01-19 15:35:53.448078: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-01-19 

tf.Tensor(
[[0.525  0.475  0.375  0.313  0.261 ]
 [0.333  0.206  0.14   0.179  0.134 ]
 [0.221  0.153  0.113  0.106  0.0671]
 ...
 [0.206  0.126  0.169  0.131  0.111 ]
 [0.256  0.155  0.173  0.181  0.132 ]
 [0.351  0.248  0.22   0.244  0.267 ]], shape=(730, 5), dtype=float64)


# ParetoGAN architecture

In [5]:
# clip model weights to a given hypercube
class ClipConstraint(Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value
 
	# clip model weights to hypercube
	def __call__(self, weights):
		return backend.clip(weights, -self.clip_value, self.clip_value)

In [6]:
# calculate wasserstein loss
def wasserstein_loss(y_true, y_pred):
	return backend.mean(y_true * y_pred)
 
# define the standalone critic model
def define_critic(in_shape=(5)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# weight constraint
	const = ClipConstraint(0.05)
	# define model
	model = tfk.models.Sequential([
 tfkl.Dense(10, activation='relu', kernel_constraint=const),
tfkl.Dense(5, activation='relu', kernel_constraint=const),
 tfkl.Dense(1,activation = 'linear')])
	opt = RMSprop(lr=0.00005)
	model.compile(loss=wasserstein_loss, optimizer=opt)
	return model
 
# define the standalone generator model
def define_generator(latent_dim):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = tfk.models.Sequential([
  tfkl.Dense(10, activation='relu'),
  tfkl.Dense(10, activation='relu'),
  tfkl.Dense(5)])
	return model
 
# define the combined generator and critic model, for updating the generator
def define_gan(generator, critic):
	# make weights in the critic not trainable
	critic.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the critic
	model.add(critic)
	# compile model
	opt = RMSprop(lr=0.00005)
	model.compile(loss=wasserstein_loss, optimizer=opt)
	return model
 
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	X = tf.convert_to_tensor(dataset.numpy()[ix])
	# generate class labels, -1 for 'real'
	y = -tf.ones((n_samples, 1))
	return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	x_input = tfd.Pareto(concentration=4.5,
    scale=1.0).sample([n_samples,latent_dim]) #if Danube experiment
	#x_input = tfd.Pareto(concentration=1.5,
    #scale=1.0).sample([n_samples,latent_dim]) #if R1 experiment
	return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels with 1.0 for 'fake'
	y = tf.ones((n_samples, 1))
	return X, y
 
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
    print("perf")
    # save the generator model
  # 	filename2 = 'model_%04d.h5' % (step+1)
  # 	g_model.save(filename2)
  # 	print('>Saved: %s and %s' % (filename1, filename2))
 
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist):
    # plot history
    plt.plot(d1_hist, label='crit_real')
    plt.plot(d2_hist, label='crit_fake')
    plt.plot(g_hist, label='gen')
    plt.legend()
    plt.savefig('plot_line_plot_loss.png')
    plt.show()
    plt.close()
 

# train the generator and critic
def train(g_model, c_model, gan_model, dataset, latent_dim, n_epochs=1000, n_batch=16, n_critic=5):
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	# lists for keeping track of loss
	c1_hist, c2_hist, g_hist = list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# update the critic more than the generator
		c1_tmp, c2_tmp = list(), list()
		for _ in range(n_critic):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update critic model weights
			c_loss1 = c_model.train_on_batch(X_real, y_real)
			c1_tmp.append(c_loss1)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update critic model weights
			c_loss2 = c_model.train_on_batch(X_fake, y_fake)
			c2_tmp.append(c_loss2)
		# store critic loss
		c1_hist.append(mean(c1_tmp))
		c2_hist.append(mean(c2_tmp))
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = -ones((n_batch, 1))
		# update the generator via the critic's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		g_hist.append(g_loss)
    
    
		if (i+1) % 50 == 0: 
			print('>%d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1_hist[-1], c2_hist[-1], g_loss))
      
      
		# evaluate the model performance every 'epoch'
		if (i+1) % 100 == 0: # %bat_per_epo
			summarize_performance(i, g_model, latent_dim)
	# line plots of loss
	plot_history(c1_hist, c2_hist, g_hist)

In [ ]:
# Training

In [ ]:
# size of the latent space
latent_dim = 5
# create the critic
critic = define_critic()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, critic)
# train model
train(generator, critic, gan_model, n_train_data, latent_dim)

>50, c1=0.048, c2=-0.066 g=0.069
>100, c1=0.057, c2=-0.064 g=0.069
perf
>150, c1=0.054, c2=-0.063 g=0.067
>200, c1=0.054, c2=-0.062 g=0.064
perf
>250, c1=0.059, c2=-0.060 g=0.062
>300, c1=0.054, c2=-0.060 g=0.059
perf
>350, c1=0.044, c2=-0.060 g=0.056
>400, c1=0.040, c2=-0.061 g=0.059
perf
>450, c1=0.051, c2=-0.056 g=0.059
>500, c1=0.049, c2=-0.059 g=0.061
perf
>550, c1=0.046, c2=-0.057 g=0.055
>600, c1=0.042, c2=-0.054 g=0.055
perf
>650, c1=0.049, c2=-0.056 g=0.058
>700, c1=0.041, c2=-0.053 g=0.057
perf
>750, c1=0.036, c2=-0.053 g=0.055
>800, c1=0.044, c2=-0.053 g=0.053
perf
>850, c1=0.039, c2=-0.053 g=0.056
>900, c1=0.043, c2=-0.052 g=0.055
perf
>950, c1=0.046, c2=-0.053 g=0.053
>1000, c1=0.041, c2=-0.051 g=0.050
perf
>1050, c1=0.040, c2=-0.052 g=0.047
>1100, c1=0.048, c2=-0.051 g=0.053
perf
>1150, c1=0.046, c2=-0.050 g=0.054
>1200, c1=0.040, c2=-0.052 g=0.049
perf
>1250, c1=0.046, c2=-0.050 g=0.053
>1300, c1=0.042, c2=-0.052 g=0.047
perf
>1350, c1=0.049, c2=-0.051 g=0.054
>1400, c1=

# Generate samples

In [13]:
N_samples = 18244
z_gen = tfd.Pareto(concentration =4.5, scale=1).sample([N_samples,5])
z     = z_gen
gen_samples = generator(z)
print(gen_samples)



tf.Tensor(
[[0.40054315 0.16265531 0.3292271  0.3286661  0.23374595]
 [0.29903626 0.19077809 0.29239857 0.00368873 0.04497734]
 [0.37850606 0.2029507  0.349822   0.32882425 0.2617416 ]
 ...
 [0.24224547 0.20328504 0.2062523  0.03195173 0.04692465]
 [0.37601614 0.15618078 0.2611254  0.3998574  0.24949239]
 [0.3971334  0.1947047  0.39811262 0.22488575 0.20972392]], shape=(18244, 5), dtype=float32)
tf.Tensor(
[[0.40054315 0.16265531 0.3292271  0.3286661  0.23374595]
 [0.29903626 0.19077809 0.29239857 0.00368873 0.04497734]
 [0.37850606 0.2029507  0.349822   0.32882425 0.2617416 ]
 ...
 [0.24224547 0.20328504 0.2062523  0.03195173 0.04692465]
 [0.37601614 0.15618078 0.2611254  0.3998574  0.24949239]
 [0.3971334  0.1947047  0.39811262 0.22488575 0.20972392]], shape=(18244, 5), dtype=float32)


# Save generator

In [13]:
filename ='/home/nlafon/These/Extreme_VAE/tmp/Exp1/GANDanube/checkpoint'
generator.save_weights(filename)

# Load generator 

In [14]:
gen2 = define_generator(5)
gen2.load_weights(filename)

2023-01-19 10:31:23.907073: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-19 10:31:23.909721: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-01-19 10:31:24.012973: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 715.34GiB/s
2023-01-19 10:31:24.013225: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:73:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.68GiB deviceMemoryBandwidth: 715.34GiB/s
2023-01-19 10:31:24.013266: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-01-19 